# Behavior stimuli

This notebook panels for Figure 2 showing examples of the stimuli used in the behavioral trials. You will need to have downloaded the `zebf-discrim-noise.zip` file from the [Zebra Finch Behavioral Discrimination of Conspecific Song in Noise](https://doi.org/10.6084/m9.figshare.27961002) dataset and unpacked it in the `datasets` folder. 

In [ ]:
import sys

sys.path.insert(0, "../scripts")

In [ ]:
from pathlib import Path

import graphics_defaults  # noqa: F401
import matplotlib.pyplot as plt
import numpy as np
from core import find_resource, load_wave, rescale, resample
from filters import SpectrogramTransform

In [ ]:
# dataset path
stim_file_path = Path("../datasets/zebf-discrim-noise/stimuli")
# spectrogram parameters
window_size = 0.010
max_frequency = 8000

In [ ]:
foreground = "9ex2k0dy"
background = "btwmt59w1"
background_levels = [-100, -65, -60, -55, -50, -45, -40, -35, -30, -25, -20]
background_levels = [-100, -60, -50, -40, -30, -20]

Color bar calibration: the foreground stimuli are normalized to an RMS amplitude of -30 dBFS. We just need
to make sure we preserve power correctly in the spectrogram transform, using the same approach as in Fig 1.

In [ ]:
dummy_stim = find_resource(f"{foreground}-30_{background}-100.wav", alt_base=stim_file_path)
dummy_signal = load_wave(dummy_stim)
stfter = SpectrogramTransform(window_size, dummy_signal["sampling_rate"], max_frequency)
fgrid = stfter.freq

In [ ]:
fig, axs = plt.subplots(nrows=len(background_levels) + 1, sharex=True, sharey=True, figsize=(1.3, 2.5), dpi=300)

for ax, level in zip(axs, background_levels):
    stim_file = stim_file_path / f"{foreground}-30_{background}{level}.wav"
    stim_data = load_wave(stim_file)
    spec = stfter.transform(stim_data["signal"], scaling=None) + 1e-6
    log_spec = 10 * np.log10(spec / stfter.scale1)
    tgrid = stfter.tgrid(spec)
    pos = ax.imshow(log_spec, aspect="auto", origin="lower", vmin=-90, vmax=-20, 
                   extent=(tgrid[0], tgrid[-1], fgrid[0] / 1000, fgrid[-1] / 1000))


# background
bkgnd_file = stim_file_path / f"{background}.wav"
bkgnd_data = load_wave(bkgnd_file)
resample(bkgnd_data, stim_data["sampling_rate"])
rescale(bkgnd_data, level)
spec = stfter.transform(bkgnd_data["signal"], scaling=None) + 1e-6
log_spec = 10 * np.log10(spec / stfter.scale1)
tgrid = stfter.tgrid(spec)
pos = axs[-1].imshow(log_spec, aspect="auto", origin="lower", vmin=-90, vmax=-20, 
               extent=(tgrid[0], tgrid[-1], fgrid[0] / 1000, fgrid[-1] / 1000))

ax.set_xlim(0, 3.0)
for ax in axs[:-1]:
    ax.get_xaxis().set_visible(False)
    ax.set_frame_on(False)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
fig.subplots_adjust(hspace=0.05, wspace=0.1)

cax = fig.add_axes((0.93, 0.11, 0.02, 0.12))
cax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
cax.yaxis.tick_right()
cbar = fig.colorbar(pos, cax=cax)


In [ ]:
fig.savefig("../figures/behavior_example_stims.pdf")